In [25]:
import os, datetime as dt, warnings, tushare as ts, pandas as pd
from dotenv import load_dotenv; load_dotenv()

# 忽略来自pandas的FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")
# 忽略来自tushare的FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="tushare")

pro = ts.pro_api(os.getenv("TUSHARE_TOKEN"))

In [28]:
ts_code = "932000.CSI"
df = ts.pro_bar(ts_code=ts_code, freq='D', asset='I', start_date='20200101', end_date='20231231', adj='qfq')

In [29]:
df

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,932000.CSI,20231229,2399.4713,2351.6999,2399.4713,2351.5501,2353.9252,45.5461,1.9349,2.163278e+08,2.374065e+08
1,932000.CSI,20231228,2353.9252,2311.1992,2361.9152,2294.0147,2315.0790,38.8462,1.6780,2.171661e+08,2.274023e+08
2,932000.CSI,20231227,2315.0790,2297.1613,2318.5108,2285.8100,2295.9741,19.1049,0.8321,1.798106e+08,1.908129e+08
3,932000.CSI,20231226,2295.9741,2330.4483,2330.4483,2289.9820,2331.3193,-35.3452,-1.5161,1.804282e+08,1.911678e+08
4,932000.CSI,20231225,2331.3193,2333.1508,2339.5616,2310.8138,2337.2158,-5.8965,-0.2523,1.797522e+08,1.844431e+08
...,...,...,...,...,...,...,...,...,...,...,...
965,932000.CSI,20200108,1899.6447,1921.8351,1928.4641,1899.6447,1928.6772,-29.0325,-1.5053,1.300200e+08,1.240955e+08
966,932000.CSI,20200107,1928.6772,1904.5751,1928.6772,1904.5751,1902.3342,26.3430,1.3848,1.210434e+08,1.179513e+08
967,932000.CSI,20200106,1902.3342,1875.6112,1910.8564,1875.6112,1886.7108,15.6234,0.8281,1.237568e+08,1.182277e+08
968,932000.CSI,20200103,1886.7108,1878.7584,1890.3723,1873.4735,1873.9215,12.7893,0.6825,1.120757e+08,1.107198e+08


In [35]:
# 重新导入模块以获取更新后的代码
import importlib
sys.path.append('/Users/feizhanxia/Documents/Heyuan/量化1期/codes/backtest_frame_v2')
from engine import data_fetcher as F
importlib.reload(F)

# 测试修复后的数据获取
ts_code = "932000.CSI"
START = "20220101"

print("=== 测试修复后的数据获取 ===")
try:
    price = F.fetch_daily(ts_code, START, dt.date.today().strftime("%Y%m%d"))
    print(f"获取数据形状: {price.shape}")
    print(f"数据是否为空: {price.empty}")
    
    if not price.empty:
        print("✅ 数据获取成功！")
        print(f"数据列: {list(price.columns)}")
        print(f"数据索引类型: {type(price.index)}")
        print(f"前3行数据:")
        print(price.head(3))
    else:
        print("❌ 数据仍然为空")
        
except Exception as e:
    print(f"获取数据时出错: {e}")
    import traceback
    traceback.print_exc()

=== 测试修复后的数据获取 ===
获取数据形状: (861, 10)
数据是否为空: False
✅ 数据获取成功！
数据列: ['ts_code', 'close', 'open', 'high', 'low', 'pre_close', 'change', 'pct_chg', 'vol', 'amount']
数据索引类型: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
前3行数据:
               ts_code      close       open       high        low  pre_close  \
trade_date                                                                      
2022-01-04  932000.CSI  2697.7209  2675.3655  2700.2387  2675.3655  2666.9693   
2022-01-05  932000.CSI  2648.9806  2697.0580  2697.0580  2624.7970  2697.7209   
2022-01-06  932000.CSI  2677.4415  2633.5138  2683.0628  2633.5138  2648.9806   

             change  pct_chg           vol        amount  
trade_date                                                
2022-01-04  30.7516   1.1531  2.454273e+08  2.488073e+08  
2022-01-05 -48.7403  -1.8067  2.508628e+08  2.613269e+08  
2022-01-06  28.4609   1.0744  2.131277e+08  2.258557e+08  
获取数据形状: (861, 10)
数据是否为空: False
✅ 数据获取成功！
数据列: ['ts_code', 'close', '